In [1]:
import numpy as np
import os
import csv
import sys
import matplotlib.pyplot as plt
import colorcet as cc
import time
import multiprocessing
import random
from tabulate import tabulate
import itertools
import os
import pickle


## __SI:__ Antimicrobial resistance level and conjugation permissiveness shape plasmid distribution in clinical enterobacteria

###6. Computer experiment: Iterative knock-out simulations
This Jupyter Notebook documents the implementation of the iterative knock-out simulations used in our study. In this approach, strains were randomly and sequentially removed from a community initially composed of 20 strains. After each removal, we re-initialized the simulation with the remaining strains, maintaining the initial total density of bacteria and the initial fraction of plasmid-bearing bacteria as in the multistrain dose-response assays.

The key measurement following each removal was the change in plasmid stability in the community, monitored through the densities of remaining strains and their plasmid-bearing fractions. This approach allowed us to identify strains whose removal significantly impacted plasmid stability, indicating that these strains formed successful associations with the plasmid, contributing to the persistence of antibiotic resistance within the community.



### Experimental parameters

In [3]:
sys.path.insert(0, 'src/')
from pOXA48_S1 import *
from pOXA48_S2 import *
from pOXA48_S3 import *

%run src/pOXA48_parameters
print_expe_params(expe_params)


Experimental Parameters:
------------------------------------------------
	Initial bacterial density (B0): 1000000.0
	Maximum drug concentration (A_max): 131072
	Antibiotic degradation rates (alphas): [1.e-10 1.e-12]
	Length of experiment (T): 24
	Initial resource concentration (S0): 1.0
	Resource decay rate (d): 0.1
	Extinction threshold: 1.0
------------------------------------------------


### Model parameters

In [4]:

model_params = import_model_params("%smodel_params.csv"%(dataPath), expe_params)
display_model_params_stats(model_params, strains_subset)


╒═════════╤═════════════╤═══════════════════════╤═════════════════════════════════╤═════╕
│ Group   │ Parameter   │ Mean                  │ Range                           │ N   │
╞═════════╪═════════════╪═══════════════════════╪═════════════════════════════════╪═════╡
│ E-TC    │ conj_rate   │ nan                   │ (nan, nan)                      │ 10  │
├─────────┼─────────────┼───────────────────────┼─────────────────────────────────┼─────┤
│ E-TC    │ VKm         │ 5.931e-10             │ (3.79e-10, 7.6e-10)             │ 10  │
├─────────┼─────────────┼───────────────────────┼─────────────────────────────────┼─────┤
│ E-TC    │ rho         │ 948158352.0           │ (550386500.0, 1121715175.0)     │ 10  │
├─────────┼─────────────┼───────────────────────┼─────────────────────────────────┼─────┤
│ E-TC    │ seg_rate    │ 0.0020000000000000005 │ (0.002, 0.002)                  │ 10  │
├─────────┼─────────────┼───────────────────────┼─────────────────────────────────┼─────┤
│ E-TC    

The computer experiments involve running a series of simulations on bacterial communities composed of varying strains and subjected to a range of antibiotic concentrations. Our methodology is aimed at evaluating how an increase in the number of strains enhances the likelihood of establishing a successful plasmid-host relationship that enables the plasmid to persist in that environment. By repeating these experiments across different community sizes and strengths of selection, we can evaluate the conditions under which a plasmid is maintained in the population.

The collected data, encapsulating the distribution of the plasmid across strains under different environmental pressures, is saved for further analysis and visualization.

In [5]:
num_days = 10
num_expe = 100
type_experiment='invasion'

In [6]:
powers = np.arange(13, 20.0, 1.0)  # array of powers of two exponents
Amax_values=[0.0] #np.power(2.0, powers)  # calculate 2 raised to each exponent

In [7]:
def simulate_environment_multistrain(model_params, istrains, E):
    """
    Simulates the bacterial dynamics for multiple strains in a specific environmental condition.

    Parameters:
    model_params: dict
        The dictionary containing model parameters.
    istrains: list
        A list of the strains that should be included in the simulation.
    E: array
        The environmental data for the simulation.

    Returns:
    Btot: array
        The total bacterial density at the end of the simulation.
    BpE, BpK: arrays
        The densities of plasmid-bearing strains with and without the resistance gene, respectively.
    BfE, BfK: arrays
        The densities of plasmid-free strains with and without the resistance gene, respectively.
    freqpE, freqpK: arrays
        The frequencies of plasmid-bearing strains with and without the resistance gene, respectively.
    times_list, ys_list: lists
        The time points and corresponding solution vectors at which the solution was evaluated.
    strains_params_list: list
        A list of the strain-specific parameters for each of the strains included in the simulation.
    """
    print('.', end="", flush=True)

    # Simulate transfers for the given strains and environment
    times_list, ys_list, strains_params_list = simulateTransfers_multistrain(model_params, istrains, E, type_experiment)

    # Extract the final time point and corresponding solution vectors
    final_times, final_ys = get_final_points(times_list, ys_list)

    # Analyze the simulation results to get bacterial densities and frequencies
    Btot, BpE, BpK, BfE, BfK, freqpE, freqpK  = analyze_simulation(model_params, istrains, final_ys)

    return Btot, BpE, BpK, BfE, BfK, freqpE, freqpK, times_list, ys_list, strains_params_list

def simulate_environments_multistrain(model_params, istrains, Es):
    """
    Simulates the bacterial dynamics for multiple strains across different environments.

    This function iterates over the given environments, simulating the dynamics of the bacterial strains
    in each environment and collecting the results.

    Parameters:
    model_params: dict
        The dictionary containing model parameters.
    istrains: list
        A list of the strains that should be included in the simulation.
    Es: list of arrays
        A list of environmental data for the simulation, each array corresponds to a different environment.

    Returns:
    Btots: list of arrays
        The total bacterial density at the end of each simulation.
    BpEs, BpKs: lists of arrays
        Lists of the densities of plasmid-bearing strains with and without the resistance gene, respectively.
    BfEs, BfKs: lists of arrays
        Lists of the densities of plasmid-free strains with and without the resistance gene, respectively.
    freqpEs, freqpKs: lists of arrays
        Lists of the frequencies of plasmid-bearing strains with and without the resistance gene, respectively.
    ts: list of lists
        Lists of the time points at which the solution was evaluated in each simulation.
    ys: list of lists
        Lists of the corresponding solution vectors for each time point in each simulation.
    params: list of lists
        Lists of the strain-specific parameters for each of the strains included in the simulation.
    """
    freqpEs = []
    freqpKs = []
    BpEs=[]
    BpKs=[]
    BfEs=[]
    BfKs=[]
    Btots=[]
    ts=[]
    ys=[]
    params=[]
    for E in Es:
        Btot, BpE, BpK, BfE, BfK, freqpE, freqpK, times_list, ys_list, params_list = simulate_environment_multistrain(model_params, istrains, E)
        freqpEs.append(freqpE)
        freqpKs.append(freqpK)
        BfEs.append(BfE)
        BfKs.append(BfK)
        BpEs.append(BpE)
        BpKs.append(BpK)
        Btots.append(Btot)
        ts.append(times_list)
        ys.append(ys_list)
        params.append(params_list)

    return Btots, BpEs, BpKs, BfEs, BfKs, freqpEs, freqpKs, ts, ys, params



In [8]:
def get_sequences_from_files(directory, length=None):
    """
    Extracts sequences of integers from the names of the files in a directory.

    This function looks for files with names that start with "sim_Es_" and end with ".pkl", then
    extracts a sequence of integers from the rest of the filename. It assumes that the integers
    are separated by underscores.

    Parameters:
    directory: str
        The path to the directory containing the files.
    length: int, optional
        The length of the sequences to look for. If specified, only sequences of this length are returned.
        If not specified, sequences of any length are returned.

    Returns:
    sequences: list of lists
        A list of the sequences that were found. Each sequence is represented as a list of integers.
    """
    sequences = []

    print(directory)
    for filename in os.listdir(directory):
        if filename.startswith("sim_Es_") and filename.endswith(".pkl"):
            sequence = filename[len("sim_Es_"):-len(".pkl")]
            sequence = list(map(int, sequence.split('_')))

            if length is None or len(sequence) == length:
                sequences.append(sequence)

    return sequences


In [9]:

def simulate_subset(model_params, istrains=[], Es=[]):
    """
    Simulates multiple bacterial strain growth for subsets of strains in varying environments.

    This function simulates bacterial growth for subsets of the given strains in the provided
    environments. It outputs the total bacterial biomass, plasmid-bearing and plasmid-free biomass
    for both environment types, and the frequency of plasmid-bearing bacteria for both environment types.

    Parameters:
    model_params: dict
        The dictionary containing model parameters.
    istrains: list, optional
        List of strain indexes to be simulated.
    Es: list, optional
        List of environments for the simulation.

    Returns:
    istrains_all, Es_all, Btots_all, BpEs_all, BpKs_all, BfEs_all, BfKs_all, freqpEs_all, freqpKs_all, ts_all, ys_all, params_all:
        Lists containing, respectively, all strain subsets, all environments, total bacterial biomass, plasmid-bearing
        biomass for both environments, plasmid-free biomass for both environments, frequency of plasmid-bearing
        bacteria for both environments, simulation times, system state for each time step, and model parameters.
    """
    freqpEs_all=[]
    freqpKs_all=[]
    Btots_all=[]
    BpEs_all=[]
    BpKs_all=[]
    BfEs_all=[]
    BfKs_all=[]
    istrains_all=[]
    Es_all=[]
    ys_all=[]
    ts_all=[]
    params_all=[]


    for subset_length in range(1, len(istrains) + 1):
      freqpEs = []
      freqpKs = []
      Btots=[]
      BpEs=[]
      BpKs=[]
      BfEs=[]
      BfKs=[]
      ys=[]
      ts=[]
      params=[]

      subset = istrains[:subset_length]
      print(f"\t{subset}: ", end="", flush=True)

      str_subset = [str(x) for x in subset]  # Convert all elements to string
      sim_lbl = "_".join(str_subset)  # Join elements with "_"
      fileName = "sim_Es_%s.pkl" % sim_lbl  # Construct the filename

      if os.path.isfile("%s%s"%(runPathN, fileName)):

        istrains_subset, Es, Btot, BpEs, BpKs, BfEs, BfKs, freqpEs, freqpKs, ts, ys, params = load_simulation_results("%s%s"%(runPathN, fileName))
        print(" Loading %s%s"%(runPathN, fileName))
      else:

        # Iterate over each E in Es
        for E in Es:

              # Run the simulation for the current E and subset
              this_Btot, this_BpEs, this_BpKs, this_BfEs, this_BfKs, this_freqpEs, this_freqpKs, this_ts, this_ys, this_params = simulate_environment_multistrain(model_params, subset, E)
              Btots.append(this_Btot)
              BpEs.append(this_BpEs)
              BpKs.append(this_BpKs)
              BfEs.append(this_BfEs)
              BfKs.append(this_BfKs)
              freqpEs.append(this_freqpEs)
              freqpKs.append(this_freqpKs)
              ts.append(this_ts)
              ys.append(this_ys)
              params.append(this_params)

              if np.any(this_freqpEs+this_freqpKs)>0.5:
                print('*', end="", flush=True)
              else:
                print('o', end="", flush=True)

        save_simulation_results("%s%s"%(runPathN, fileName), subset, Es, Btots, BpEs, BpKs, BfEs, BfKs, freqpEs, freqpKs, ts, ys, params)
        print(" Saving %s%s"%(runPathN,fileName))

      istrains_all.append(subset)
      Btots_all.append(Btots)
      Es_all.append(Es)
      BpEs_all.append(BpEs)
      BpKs_all.append(BpKs)
      freqpEs_all.append(freqpEs)
      ts_all.append(ts)
      ys_all.append(ys)
      params_all.append(params)

    return istrains_all, Es_all, Btots_all, BpEs_all, BpKs_all, BfEs_all, BfKs_all, freqpEs_all, freqpKs_all, ts_all, ys_all, params_all


In [10]:
def select_random_elements(strains_subset, num_elements):
    """
    Selects a specified number of random elements from a given list.

    This function randomly selects a specified number of elements from the provided list,
    without replacement. The function is primarily used for selecting a subset of bacterial strains.

    Parameters:
    strains_subset: list
        List from which elements are to be randomly selected.
    num_elements: int
        Number of elements to be randomly selected from the list.

    Returns:
    random_elements: list
        List of randomly selected elements.
    """

    random_elements = random.sample(strains_subset, num_elements)
    return random_elements


In [11]:
def run_experiment(model_params, strains_subset, num_expe, istrains_done):
    """
    Runs the entire simulation experiment for a given set of strains.

    This function performs a specified number of simulation experiments on a provided subset of strains.
    Each experiment runs a series of simulations using a set of strains chosen either from a list of
    already completed strains (if available) or randomly selected from the subset.

    Parameters:
    model_params: dict
        The dictionary containing model parameters.
    strains_subset: list
        List containing all potential strains that can be used in the simulation.
    num_expe: int
        Number of simulation experiments to be performed.
    istrains_done: list
        List of strain sets for which simulations have already been completed.

    Returns:
    Various lists of experiment results including total biomass, total plasmid-bearing and
    plasmid-free strains, total plasmid frequency, specific time points, states at those time
    points, and strain-specific parameters used.
    """

    max_strains = len(strains_subset)

    istrains_expe = []
    Es_expe = []
    Btots_expe = []
    BpEs_expe = []
    BpKs_expe = []
    BfEs_expe = []
    BfKs_expe = []
    freqpEs_expe = []
    freqpKs_expe = []
    ts_expe = []
    ys_expe = []
    params_expe = []

    for i in range(num_expe):
        start_time = time.time()  # Record the start time

        if i < len(istrains_done):
            istrains = istrains_done[i]
            print('* ', i, ': istrains =', istrains)
        else:
            istrains = select_random_elements(strains_subset, max_strains)
            print('** ', i, ': istrains =', istrains)

        # Run the simulations
        istrains_subset, Es_subset, Btots_subset, BpEs_subset, BpKs_subset, BfEs_subset, BfKs_subset, freqpEs_subset, freqpKs_subset, ts_subset, ys_subset, params_subset = simulate_subset(model_params, istrains, As)

        istrains_expe.append(istrains_subset)
        Es_expe.append(Es_subset)
        Btots_expe.append(Btots_subset)
        BpEs_expe.append(BpEs_subset)
        BpKs_expe.append(BpKs_subset)
        BfEs_expe.append(BfEs_subset)
        BfKs_expe.append(BfKs_subset)
        freqpEs_expe.append(freqpEs_subset)
        freqpKs_expe.append(freqpKs_subset)
        ts_expe.append(ts_subset)
        ys_expe.append(ys_subset)
        params_expe.append(params_subset)

        end_time = time.time()  # Record the end time
        elapsed_time = end_time - start_time  # Calculate the elapsed time

        print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    return istrains_expe, Es_expe, Btots_expe, BpEs_expe, BpKs_expe, BfEs_expe, BfKs_expe, freqpEs_expe, freqpKs_expe, ts_expe, ys_expe, params_expe



In [12]:
import math

As_norm=np.ones(num_days)
for Amax in Amax_values:
  #print("******** Amax=2**%s"%math.log2(Amax))

  As = [[Amax * e for e in As_norm]]

  expeLabel='%s_N%s_A%se-2'%(str_E, num_days, int(Amax*100))
  runPathN = path+'runs/N'+str(num_days)+'/'+expeLabel+'/'
  print("runPath=%s"%runPathN)


  if not os.path.exists(runPathN):
    os.makedirs(runPathN)

  istrains_done = get_sequences_from_files(runPathN, length=len(strains_subset))
  istrains_done = istrains_done[:num_expe]

  print(istrains_done)


  istrains_expe, Es_expe, Btots_expe, BpEs_expe, BpKs_expe, BfEs_expe, BfKs_expe, freqpEs_expe, freqpKs_expe, ts_expe, ys_expe, params_expe = run_experiment(model_params, strains_subset, num_expe, istrains_done)


runPath=./runs/N10/constant_N10_A0e-2/
./runs/N10/constant_N10_A0e-2/
[[18, 11, 37, 45, 15, 41, 43, 2, 29, 34, 26, 20, 21, 39, 38, 1, 25, 6, 9, 24], [43, 29, 34, 11, 2, 20, 41, 25, 1, 18, 38, 45, 24, 15, 37, 9, 39, 21, 26, 6], [26, 41, 38, 15, 20, 6, 21, 37, 9, 43, 34, 18, 11, 24, 39, 25, 1, 29, 2, 45], [45, 41, 18, 2, 21, 26, 29, 9, 20, 11, 34, 24, 1, 6, 38, 37, 15, 43, 39, 25], [29, 34, 41, 45, 39, 9, 15, 2, 24, 6, 18, 38, 43, 37, 20, 26, 1, 11, 25, 21], [43, 21, 34, 2, 41, 25, 37, 39, 9, 1, 20, 29, 26, 15, 45, 11, 24, 6, 38, 18], [21, 41, 2, 37, 26, 9, 34, 39, 45, 38, 18, 24, 43, 11, 20, 15, 1, 29, 25, 6], [41, 21, 39, 38, 43, 34, 1, 25, 15, 24, 6, 18, 29, 11, 2, 20, 26, 9, 37, 45], [20, 2, 11, 29, 25, 38, 6, 9, 41, 1, 37, 24, 39, 26, 18, 34, 15, 21, 45, 43], [2, 6, 37, 21, 26, 20, 11, 1, 18, 15, 38, 41, 45, 24, 29, 9, 43, 25, 34, 39], [37, 26, 2, 38, 15, 39, 25, 20, 1, 45, 21, 34, 41, 6, 9, 29, 11, 24, 18, 43], [37, 11, 29, 6, 45, 15, 20, 18, 38, 41, 21, 34, 25, 39, 2, 43, 26, 1, 9